In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

2024-08-21 16:42:01.528780: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-21 16:42:01.532074: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-21 16:42:01.542776: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 16:42:01.559845: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 16:42:01.564611: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-21 16:42:01.575880: I tensorflow/core/platform/cpu_feature_gu

In [2]:
def parse_tfrecord_fn(record):
    feature_description = {
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/filename': tf.io.FixedLenFeature([], tf.string),
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64)
    }

    parsed_features = tf.io.parse_single_example(record, feature_description)

    # Decode and preprocess the image
    image = tf.image.decode_jpeg(parsed_features['image/encoded'], channels=3)
    image = tf.image.resize(image, [224, 224])  # Adjust size as needed
    image = tf.cast(image, tf.float32) / 255.0  # Normalize
    # Image should have shape [224, 224, 3] after this point

    # Convert sparse tensors to dense tensors for bounding boxes
    xmin = tf.sparse.to_dense(parsed_features['image/object/bbox/xmin'])
    xmax = tf.sparse.to_dense(parsed_features['image/object/bbox/xmax'])
    ymin = tf.sparse.to_dense(parsed_features['image/object/bbox/ymin'])
    ymax = tf.sparse.to_dense(parsed_features['image/object/bbox/ymax'])
    labels = tf.sparse.to_dense(parsed_features['image/object/class/label'], default_value=-1)

    # Stack bounding box coordinates into a 2D tensor [num_boxes, 4]
    bbox = tf.stack([xmin, xmax, ymin, ymax], axis=-1)

    # Padding to ensure consistent shape across batches
    max_objects = 10  # Adjust this value based on your dataset
    bbox = tf.image.pad_to_bounding_box(bbox, 0, 0, max_objects, 4)
    labels = tf.pad(labels, [[0, max_objects - tf.shape(labels)[0]]], constant_values=-1)

    return image, (bbox, labels)



def load_data(filename, batch_size=32):
    dataset = tf.data.TFRecordDataset(filename)
    dataset = dataset.map(parse_tfrecord_fn)
    dataset = dataset.shuffle(buffer_size=1000)  # Shuffle data
    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

train_dataset = load_data('/home/tugalinebacker/hms_surprise/data/raw/first_shot/train.record')
test_dataset = load_data('/home/tugalinebacker/hms_surprise/data/raw/first_shot/test.record')


I0000 00:00:1724254924.450932  140633 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-21 16:42:04.451793: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


ValueError: in user code:

    File "/tmp/ipykernel_140633/250037283.py", line 34, in parse_tfrecord_fn  *
        bbox = tf.image.pad_to_bounding_box(bbox, 0, 0, max_objects, 4)

    ValueError: 'image' (shape (None, 4)) must have either 3 or 4 dimensions.


In [3]:
def build_object_detection_model(num_classes):
    inputs = tf.keras.Input(shape=(224, 224, 3))  # Adjust shape as needed
    
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    
    bbox_output = tf.keras.layers.Dense(4, activation='sigmoid', name='bbox_output')(x)  # Bounding box coordinates
    class_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='class_output')(x)  # Class predictions
    
    model = tf.keras.Model(inputs, [bbox_output, class_output])
    
    model.compile(optimizer='adam',
                  loss={'bbox_output': 'mean_squared_error',
                        'class_output': 'categorical_crossentropy'},
                  metrics={'bbox_output': 'mae', 'class_output': 'accuracy'})
    
    return model

num_classes = 3
model = build_object_detection_model(num_classes)


2024-08-21 16:38:39.855064: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-08-21 16:38:39.977751: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-08-21 16:38:40.035159: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


In [4]:
model.fit(train_dataset, epochs=10, validation_data=test_dataset)


Epoch 1/10


ValueError: None values not supported.